In [4]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

In [5]:


sess = sage.Session()

WORK_DIRECTORY = "./data"

# S3 prefix
prefix = "demo"

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [6]:
hyperparameters = {
    "task" : "tasd", 
    "dataset" : "shulex", 
    "model_name_or_path" : "t5-base", 
    "eval_batch_size" :"16",
    "train_batch_size" :"2",
    "learning_rate" :"3e-4",
    "num_train_epochs":"3",
    "n_gpu": "1"
}

In [5]:
entry_point = 'finetune_prompt.py'
source_dir = './'
git_config = None
role = get_execution_role()
framework_version = '1.7.1'
py_version='py36'
instance_type='ml.p3.2xlarge'
instance_count=1

In [28]:
estimator = PyTorch(
    entry_point = entry_point,
    source_dir = source_dir,
    git_config = git_config,
    role = role,
    debugger_hook_config=False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

In [29]:
inputs = {'tasd': data_location+'/tasd/'}

In [30]:
response = estimator.fit(inputs)

2022-10-20 05:55:41 Starting - Starting the training job...
2022-10-20 05:56:05 Starting - Preparing the instances for trainingProfilerReport-1666245341: InProgress
.........
2022-10-20 05:57:39 Downloading - Downloading input data
2022-10-20 05:57:39 Training - Downloading the training image.....................
2022-10-20 06:01:05 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-20 06:01:08,537 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-20 06:01:08,567 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-20 06:01:08,576 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-20 06:01:09,257 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requir

In [7]:
import sagemaker

instance_type = 'ml.m5.4xlarge'
# instance_type='local'
role = sagemaker.get_execution_role()

In [8]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://sagemaker-us-west-2-064542430558/pytorch-training-2022-10-20-05-55-39-520/output/model.tar.gz',#estimator.model_data, 
                             role=role,
                             entry_point='inference-prompt.py', 
                             source_dir='./', 
                             framework_version='1.7.1', 
                             py_version='py36'
                ) # TODO set model_server_workers=1 to avoid torchhub bug

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

--------------!

In [10]:
%%time

predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

body = {"inputs": "I am pretty new to pickleball and finally decided to try out some different paddles."}

predictor.predict(body,initial_args={"ContentType":"application/json"})

CPU times: user 13.5 ms, sys: 0 ns, total: 13.5 ms
Wall time: 966 ms


{'result': 'Features&Product Experience/Structure/Easy tipping is great because pickleball is great'}